In [52]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

https://github.com/ironhack-labs/lab-imbalanced-data

In [53]:
churn_d = pd.read_csv('customer_churn.csv')

Load the dataset and explore the variables.

In [23]:
churn_d.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


2.We will try to predict variable Churn using a logistic regression on variables tenure, SeniorCitizen,MonthlyCharges.


pay attention to the fact that no is much larger than yes 

In [24]:
churn_d['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

- With highly skewed data like this you want to avoid the model getting a bias 
- to avoid this you want the same number of cases for each type 

# 5.regular logistic regression

In [25]:
#scale the data 
numericData=churn_d[['tenure','SeniorCitizen','MonthlyCharges']]
Y = pd.DataFrame(data=churn_d,columns=['Churn'])
transformer = StandardScaler().fit(churn_d[['tenure','SeniorCitizen','MonthlyCharges']])
scaled_x=transformer.transform(churn_d[['tenure','SeniorCitizen','MonthlyCharges']])
#----------------------------------------------------------------------
#apply the logistic regression
Classification= LogisticRegression(random_state=0,solver='lbfgs'
                                   ,multi_class='ovr').fit(scaled_x,churn_d['Churn'])
# there are different solvers you can apply to your logisticregression 
    # check how they hold up 
Classification.score(scaled_x, churn_d['Churn'])

# do a train test split ** then calculate the accuracy 
 

0.7911401391452506

Notice that even with the imbalance when we tried to fit the model without any data cleaning we still got nearly 80%

this means that the accuracy dosnt isnt always the most accurate measure 

In [26]:
5174/(5174+1869) # this is the accuracy of the model


0.7346301292063041

In [27]:
#filter yes and no into 2 variables 
yes = churn_d[churn_d['Churn']=='Yes']
no = churn_d[churn_d['Churn']=='No']

In [28]:
churn_d['Churn'].value_counts() 
#we will take yes and increase the number of samples 

No     5174
Yes    1869
Name: Churn, dtype: int64

# undersampling 

In [29]:

data1=yes
datas=no.sample(churn_d['Churn'].value_counts()[1]) 
#the sample function is used  
    #https://www.geeksforgeeks.org/python-random-sample-function/
#sample() is an inbuilt function of random module in Python that 
    #returns a particular length list of items chosen from the 
    #sequence i.e. list, tuple, string or set. Used for random 
    #sampling without replacement.  
    
#no.sample focuses on the no's in churn
    
#so we will pick random points from the churn columns 
#how many data points we want will be denoted by 
    #.value_counts()[1] 

In [30]:
data_final=pd.concat([data1,datas],axis=0) 
#add the 2 dataframes together the original data1 and the new datas


In [31]:
data_final.shape #these are the random points selected to balance out yes

(3738, 21)

# Random Over sampling 

increase the yes to match the no 

In [32]:
churn_d['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [33]:
yes=churn_d[churn_d['Churn']=='Yes']
no = churn_d[churn_d['Churn']=='No']

In [34]:
data3 = yes.sample(5174,replace = True)  
# randomly sample data from the same box aka the yes box
#similar to bootstraping  

# Question is 5174 --> N?

#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html
    #replace :bool, default False --> Allow or disallow sampling of the same row more than once.

In [35]:
data3.shape

(5174, 21)

In [36]:
data3.drop_duplicates().shape

(1747, 21)

In [37]:
data3.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
2812,3763-GCZHZ,Male,0,Yes,No,66,Yes,Yes,Fiber optic,No,Yes,No,Yes,Yes,Yes,One year,Yes,Electronic check,104.05,6890,Yes
3079,2141-RRYGO,Female,0,No,No,4,Yes,No,DSL,No,Yes,No,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),68.65,261.25,Yes
3906,6372-RFVNS,Female,0,Yes,No,1,Yes,No,Fiber optic,No,No,No,No,Yes,No,Month-to-month,No,Electronic check,78.80,78.8,Yes
354,0623-IIHUG,Female,1,No,No,21,No,No phone service,DSL,Yes,No,No,No,Yes,No,Month-to-month,Yes,Electronic check,41.90,840.1,Yes
1054,5981-ITEMU,Male,0,Yes,No,40,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),97.10,3706.95,Yes


In [38]:
no.sample(frac=0.5).shape
#this is half of the original data for no 
#this is undersampling  
#no originally had 5174 rows 


(2587, 21)

# Or 

- you can also limit the sample size directly with a number instead of a fraction

In [39]:
no.sample(100).shape  
#not preferred method 


(100, 21)

In [40]:
churn_d['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [41]:
#no.sample(churnData['churn'].value_counts()[1]).shape
# you want them both to have 1869 rows and 21 columns  
# want equal quantities of yes and equal quantities of no 

# Increaseing the Imbalance  
- to see how the model works  
    - lower the yes data points to 500 

In [42]:

yes = churn_d[churn_d['Churn']=='Yes']
no = churn_d[churn_d['Churn']== 'No']
yes = yes.sample(500) 

#reducing the sample that is over represented 
#or you could undersample the category in excess by taking a random subset of the data provided 
#over sample- the under represented categroy  

# your accuracy will still be high, but the model would not be a good representaion of the data set becasue of the imbalance will result in the model being bias and the majority class being overrepresented  

In [43]:
data = pd.concat([yes,no],axis=0)
print(data['Churn'].value_counts())
data.head()


No     5174
Yes     500
Name: Churn, dtype: int64


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
6205,7547-EKNFS,Male,0,Yes,No,42,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Bank transfer (automatic),95.25,3959.35,Yes
6137,0870-VEMYL,Female,0,No,No,5,Yes,No,DSL,Yes,No,No,Yes,No,No,Month-to-month,No,Credit card (automatic),53.85,259.8,Yes
4960,7480-QNVZJ,Male,1,No,No,50,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,95.70,4816.7,Yes
5565,5087-SUURX,Female,0,Yes,No,18,No,No phone service,DSL,No,No,Yes,No,Yes,No,Month-to-month,Yes,Electronic check,39.05,669.85,Yes
1686,6839-ITVZJ,Female,0,Yes,Yes,26,Yes,Yes,DSL,No,Yes,No,Yes,No,No,Month-to-month,Yes,Electronic check,60.05,1616.15,Yes


# Shuffling data 

In [44]:
data= data.sample(frac=1) 
data['Churn'].value_counts()

No     5174
Yes     500
Name: Churn, dtype: int64

after the rebalancing 

In [45]:
numericData = data[['tenure','SeniorCitizen','MonthlyCharges']]
transformer = StandardScaler().fit(data[['tenure','SeniorCitizen','MonthlyCharges']])
scaled_x = transformer.transform(data[['tenure','SeniorCitizen','MonthlyCharges']])

classification=LogisticRegression(random_state=0, solver = 'lbfgs',
                  multi_class = 'ovr').fit(scaled_x, data['Churn'])
classification.score(scaled_x,data['Churn'])

0.9122312301727177

Despite the advantage of balancing classes, these techniques also have their weaknesses (there is no free lunch). The simplest implementation of over-sampling is to duplicate random records from the minority class, which can cause overfitting. In under-sampling, the simplest technique involves removing random records from the majority class, which can cause loss of information.

In [46]:
counts = churn_d['Churn'].value_counts()
yes = churn_d[churn_d['Churn']== 'Yes'].sample(counts[0],replace= True)
no = churn_d[churn_d['Churn']== 'No'] 
data = pd.concat([yes,no],axis=0)
data = data.sample(frac=1) #used for both over sampling and undersampling 
data['Churn'].value_counts()

No     5174
Yes    5174
Name: Churn, dtype: int64

In [47]:
counts = churn_d['Churn'].value_counts() 
counts

No     5174
Yes    1869
Name: Churn, dtype: int64

In [51]:
X = data[['tenure','SeniorCitizen','MonthlyCharges']]
transformer = StandardScaler().fit(X)
X = transformer.transform(X)

calssification = LogisticRegression(random_state=0,solver='lbfgs',
                                   multi_class='ovr').fit(X, data['Churn'])
classification.score(X, data['Churn']) 


# more reliable than the .7 we had before on the imbalnaced data 

0.5002899110939312

 Other more sophisticated resapling techniques include:

Cluster the records of the majority class, and do the under-sampling by removing records from each cluster, thus seeking to preserve information. 
In over-sampling, instead of creating exact copies of the minority class records, we can introduce small variations into those copies, creating more diverse synthetic samples.

## use pip install imblearn 

In [49]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


# Random Under Sampler

In [51]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [53]:
rus = RandomUnderSampler()
X = churn_d[['tenure','SeniorCitizen','MonthlyCharges']]
transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = churn_d['Churn']
X_rus,y_rus = rus.fit_resample(X,y) 
#resample is the correct format for this 

In [54]:
y.value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [56]:
y_rus.value_counts()

Yes    1869
No     1869
Name: Churn, dtype: int64

In [62]:
transformer = StandardScaler().fit(X_rus)
X = transformer.transform(X_rus)
classification = LogisticRegression(random_state=0, solver='lbfgs',
                        multi_class='ovr').fit(X, y_rus)
classification.score(X, y_rus)

0.7255216693418941

# Random Over sampler 

In [ ]:
#dont have to check for the imbalance 
# if the skew is huge, if you undersample you will loose almost all of your data 
# if you undersample 
    #ask yourself how much data are you willing to lose

In [64]:
ros = RandomOverSampler()
X = churn_d[['tenure','SeniorCitizen','MonthlyCharges']]
transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = churn_d['Churn']
X_ros,y_ros = ros.fit_resample(X,y)


In [65]:
y.value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [66]:
y_ros.value_counts()


No     5174
Yes    5174
Name: Churn, dtype: int64

In [69]:
transformer = StandardScaler().fit(X_ros)
X = transformer.transform(X_ros)
classification = LogisticRegression(random_state=0, solver='lbfgs',
                        multi_class='ovr').fit(X, y_ros)
classification.score(X, y_ros)

0.7347313490529571

In [ ]:
# k and n can be used to solve prediction problems 
# can also be used to oversample the data 
# k&n uses a distance metric , it tries to find the nearest points to it 
    #applies this to all the rows, aka all the points 

# Synthetic Minorty Oversampling Technique (SMOTE)

Synthetic Minority Oversampling TEchnique (SMOTE) is an over sampling technique based on nearest neighbors that adds new points between existing points. Apply imblearn.over_sampling.SMOTE to the dataset. Build and evaluate the logistic regression model. Is it there any improvement?

In [ ]:
#is this the correct code ?? imblearn.over_sampling.SMOTE??

In [ ]:
The SMOTE algorithm can be broken down into foll. steps:
Randomly pick a point from the minority class.
Compute the k-nearest neighbors (for some pre-specified k) for this point.
Add k new points somewhere between the chosen point and each of its neighbors.

In [56]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X = churn_d[['tenure', 'SeniorCitizen','MonthlyCharges']] # select you x variables, all numerical need to be scaled 
transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = churn_d['Churn'] 

X_sm, y_sm = smote.fit_resample(X, y) #automatically runs the algorithm under the hood
y_sm.value_counts()  





#MUST USE FIT_(RE)SAMPLE OR IT WONT WORK 

#these points are not repitions are new point made close to points that already exist 

No     5174
Yes    5174
Name: Churn, dtype: int64

 # now apply 
    - the smote altered data and check the model 

In [58]:
#transformer = StandardScaler().fit(X_ros)
#X = transformer.transform(X_ros) 
classification = LogisticRegression(random_state=0, solver='lbfgs',
                        multi_class='ovr').fit(X_sm, y_sm)
classification.score(X_sm, y_sm)

0.734441437959026

# UnderSampling using TomekLinks 

Tomek links are pairs of very close instances, but of opposite classes. Removing the instances of the majority class of each pair increases the space between the two classes, facilitating the classification process.

In [ ]:
#some points are removed, losing key points, if the underpresented class is really low you lose to much data 

In [60]:
from imblearn.under_sampling import TomekLinks

tl = TomekLinks('majority')
X_tl, y_tl = tl.fit_resample(X, y)
y_tl.value_counts()

No     4694
Yes    1869
Name: Churn, dtype: int64

In [61]:
X_tl2, y_tl2 = tl.fit_resample(X_tl, y_tl)
y_tl2.value_counts()

No     4537
Yes    1869
Name: Churn, dtype: int64

Apply logistic regression to TomekLinks altered data 

In [62]:
#help ? doing the logistic regession with the correct variables ??
classification = LogisticRegression(random_state=0, solver='lbfgs',
                        multi_class='ovr').fit(X_tl2, y_tl2)
classification.score(X_tl2, y_tl2)

0.7939431782703715

In [ ]:
# It does not make the two classes equal but only removes the points from the majority 
# class that are close to other poitns in minority class

In [ ]:
# Similar way it works for multi label classification models 
# But it can be a little tricky 

# works for multi label classification models 

In [81]:
X = churn_d[['tenure', 'SeniorCitizen','MonthlyCharges']]
transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = churn_d['Churn']

In [82]:
y[6000:7000] = 'Hello'
y.value_counts() 

No       4445
Yes      1598
Hello    1000
Name: Churn, dtype: int64

In [84]:
tl = TomekLinks('majority')
X_tl, y_tl = tl.fit_resample(X, y)
y_tl.value_counts()

No       3747
Yes      1598
Hello    1000
Name: Churn, dtype: int64

In [86]:
X_sm, y_sm = smote.fit_resample(X, y)
y_sm.value_counts()

No       4445
Hello    4445
Yes      4445
Name: Churn, dtype: int64

In [ ]:
Smote and tomeklinks are used to lower the imbalance 
